In [ ]:
!pip install pydub
!pip install pympi-ling

import pandas as pd
import pympi
import numpy as np
import re
from collections import Counter
from pydub import AudioSegment
from google.colab import sheets

In [12]:
#determine overlaps for all tiers of ELAN subtitles

def process_elan_file(elan_file_path):
  #next step: create a method for creating the overlap list for all the speakers in the conversation
  #takes in ELAN file, returns a DataFrame with the overlaps in format (start_time, end_time)
  def get_overlaps(eaf_file):
    eaf = pympi.Elan.Eaf(eaf_file)
    overlaps = pd.DataFrame()
    #iterate over tiers
    for tier_a in eaf.get_tier_names():
      if 'utt' in tier_a:
        for tier_b in eaf.get_tier_names():
          if 'utt' in tier_b and tier_a != tier_b:
            overlaps_temp = pd.DataFrame(list(eaf.get_gaps_and_overlaps2(tier_a, tier_b, maxlen=-1)))
            overlaps_temp = overlaps_temp[overlaps_temp[2] == 'O12']
            overlaps = pd.concat([overlaps, overlaps_temp])

    overlaps = overlaps.drop(columns=[2])
    overlaps.columns = ['start_time1', 'end_time1']
    return overlaps

  overlap_times = get_overlaps(elan_file_path)
  #print(overlap_times)

  #get average length of overlaps
  length = 0
  for index, row in overlap_times.iterrows():
    length += row['end_time1'] - row['start_time1']

  print("avg duration of overlap =", length/len(overlap_times), "ms")

  #get word list from overlapping sections of ELAN file

  def get_words(txt_file, overlap_times, window_size):
    text = pd.read_csv(txt_file, sep='\t')
    words = pd.DataFrame()
    for index, row in overlap_times.iterrows():
      start_time = row['start_time1']
      end_time = row['end_time1']
      words_temp = text[(text['start_time1'] >= start_time - window_size) & (text['end_time1'] <= end_time + window_size)]
      words = pd.concat([words, words_temp])
      words.drop_duplicates(inplace=True)
    words_final = words['content']
    return words_final

  txt_file_path = elan_file_path.replace('.eaf', '.txt')
  word_list_initial = get_words(txt_file_path, overlap_times, 100)

  #tokenize word list
  word_list = word_list_initial.to_string().split()

  #count frequency of individual words, dropping single occurrence items
  #occurrence = {item: word_list.count(item) for item in word_list}
  occurrence = Counter(word_list)


  for word in word_list:
    if occurrence.get(word) == 1:
      word_list.remove(word)

  word_series = pd.DataFrame({
      'Word': word_list,
      'Frequency': [occurrence[word] for word in word_list]
    })

  #filter non word elements
  def is_word(text):
        pattern = re.compile(r'^[a-zA-Z]+$')
        return bool(pattern.match(text))

  df_filtered = word_series[word_series['Word'].apply(is_word)]

  #compare with average rate of occurrence of word in the entire conversation, to see which words appear with higher frequency around moments of overlap
  #to compare them, creating a column with frequency_overlaps/frequency_overall

  with open(txt_file_path, 'r') as text:
    complete_wordlist = text.read().split()
    total_occurrence = {item: complete_wordlist.count(item) for item in complete_wordlist}
      # Calculate relative frequency
    df_filtered['Relative Frequency'] = df_filtered['Word'].apply(lambda word: df_filtered[df_filtered['Word'] == word]['Frequency'].values[0] / total_occurrence.get(word, 1))


  df_filtered.sort_values(by='Relative Frequency', ascending=False, inplace=True)
  df_filtered.drop_duplicates(subset='Word', keep='first', inplace=True)
  return df_filtered.head(200)

# Example usage:
elan_files = ['/content/S01.eaf', '/content/S02.eaf' , '/content/S03.eaf', '/content/S04.eaf', '/content/S05.eaf', '/content/S06.eaf', '/content/S07.eaf', '/content/S08.eaf', '/content/S09.eaf', '/content/S10.eaf']
# Replace with your actual file paths
results = {}
for elan_file in elan_files:
  results[elan_file] = process_elan_file(elan_file)

# Print or further process the results as needed
#for file, result in results.items():
  #print(f"Results for {file}:\n{result}\n")


avg duration of overlap = 3203.6234458259323 ms


<ipython-input-12-af2679f10432>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Relative Frequency'] = df_filtered['Word'].apply(lambda word: df_filtered[df_filtered['Word'] == word]['Frequency'].values[0] / total_occurrence.get(word, 1))
<ipython-input-12-af2679f10432>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.sort_values(by='Relative Frequency', ascending=False, inplace=True)
<ipython-input-12-af2679f10432>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

avg duration of overlap = 4559.127725856698 ms


<ipython-input-12-af2679f10432>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Relative Frequency'] = df_filtered['Word'].apply(lambda word: df_filtered[df_filtered['Word'] == word]['Frequency'].values[0] / total_occurrence.get(word, 1))
<ipython-input-12-af2679f10432>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.sort_values(by='Relative Frequency', ascending=False, inplace=True)
<ipython-input-12-af2679f10432>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

avg duration of overlap = 4200.461538461538 ms


...

<ipython-input-12-af2679f10432>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Relative Frequency'] = df_filtered['Word'].apply(lambda word: df_filtered[df_filtered['Word'] == word]['Frequency'].values[0] / total_occurrence.get(word, 1))
<ipython-input-12-af2679f10432>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.sort_values(by='Relative Frequency', ascending=False, inplace=True)
<ipython-input-12-af2679f10432>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

avg duration of overlap = 3385.9650259067357 ms


<ipython-input-12-af2679f10432>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Relative Frequency'] = df_filtered['Word'].apply(lambda word: df_filtered[df_filtered['Word'] == word]['Frequency'].values[0] / total_occurrence.get(word, 1))
<ipython-input-12-af2679f10432>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.sort_values(by='Relative Frequency', ascending=False, inplace=True)
<ipython-input-12-af2679f10432>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

avg duration of overlap = 5837.74182444062 ms


<ipython-input-12-af2679f10432>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Relative Frequency'] = df_filtered['Word'].apply(lambda word: df_filtered[df_filtered['Word'] == word]['Frequency'].values[0] / total_occurrence.get(word, 1))
<ipython-input-12-af2679f10432>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.sort_values(by='Relative Frequency', ascending=False, inplace=True)
<ipython-input-12-af2679f10432>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

In [13]:
# Concatenate the results into a single DataFrame
all_data = pd.concat([result for result in results.values()], ignore_index=True)

# Group by 'Word' and sum the frequencies
merged_df = all_data.groupby('Word').agg({'Frequency': 'sum'}).reset_index()

# Calculate total word count across both files
total_word_count = sum([len(text.read().split()) for text in [open(file.replace('.eaf', '.txt'), 'r') for file in elan_files]])

# Calculate relative frequency
merged_df['Relative Frequency'] = merged_df['Frequency'] / total_word_count * 100

sheet = sheets.InteractiveSheet(df=merged_df)

https://docs.google.com/spreadsheets/d/1ObRgyISqXj40EmKV1rR1Ku8ctHBxOovorE2I8nQSqg0#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})
